<a href="https://colab.research.google.com/github/upasana-crypto/EEG/blob/main/siezure_good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [2]:
!pip install pyedflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 29.7 MB/s 


In [2]:
import pyedflib
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import warnings 
import random
import collections
from PIL import Image
from numpy.random import seed
import datetime
from time import mktime
from time import gmtime, strptime


In [3]:
import warnings
warnings.filterwarnings("ignore")
pathDataSet = '/content/drive/MyDrive/upsi-seizure/data/chb08/'
patients = ["08"]

In [4]:
def loadSummaryPatient(index):
  f = open(pathDataSet+'/chb'+patients[index]+'-summary (1).txt', 'r') #change to summary.txt
  parent = '/content/drive/MyDrive/upsi-seizure/data/chb'+patients[index]+'/'
  return f, parent

In [5]:
def createDataset():
    
    seizure_trials = []
    total_duration = 0
    print("START \n")
    for indexPatient in range(0, len(patients)):
        # fileList = []
    
        f, parent = loadSummaryPatient(indexPatient)
        line=f.readline()
        while (line):
            data=line.split(':')
            if (data[0]=="File Name"):
                name_F=data[1].strip()
                print(name_F)
                fileSt=f.readline().split(': ')[1]
                Sthr=fileSt.split(':')[0]
                #print(Sthr)
                Stmin=fileSt.split(':')[1]
                #print(Stmin)
                Stsec=fileSt.split(':')[2]
                #print(Stsec)
                fileEn=f.readline().split(': ')[1]
                Enhr=fileEn.split(':')[0]
                #print(Enhr)
                Enmin=fileSt.split(':')[1]
                #print(Enmin)
                Ensec=fileSt.split(':')[2]
                #print(Ensec)
                trial_duration = ((int(Enhr) - int(Sthr))*3600 + (int(Enmin) - int(Stmin))*60 + (int(Ensec) - int(Stsec)))
                for i in range(1):
                    line=f.readline()
                for j in range(0, int(line.split(': ')[1])):
                    #if j==0:
                       # continue
                    #else:
                    seizure_trials.append(name_F)
                    total_duration += trial_duration
                    secSt=int(f.readline().split(': ')[1].split(' ')[0])
                    print(secSt)
                    secEn=int(f.readline().split(': ')[1].split(' ')[0])
                    print(secEn)
                    seizureImageGenerate(secSt, secEn, name_F, parent)
            line=f.readline()
        f.close()

    print("END \n")
    return seizure_trials, total_duration

In [6]:
def seizureImageGenerate(secSt, secEn, name_F, parent):
  file1 = pyedflib.EdfReader(pathDataSet+name_F)
  n = file1.signals_in_file
  # print(n)
  signal_labels = file1.getSignalLabels()
  signal_headers = file1.getSignalHeaders()
  rate = signal_headers[0]['sample_rate']
  dur = file1.getFileDuration()
  print(dur)
  x = np.zeros((n, file1.getNSamples()[0]))
  for i in range(n):
    x[i,:] = file1.readSignal(i)
    # print(x)
    label = file1.getLabel(i)
  file1.close()
  #x_filter = butter_bandpass_filter(x ,lowcut , highcut , fs , order = 5)
  #a = os.getcwd()
  path = '/content/drive/MyDrive/upsi-seizure/test1/'
  if os.path.isdir(path) is not True:
    os.makedirs(path)
  if os.path.isdir(path + name_F.split('.')[0] + '/') is not True:
    os.makedirs(path + name_F.split('.')[0] + '/')
  print(path + name_F.split('.')[0] + '/')
  picnum = int(dur*rate/256)
  print(picnum)
  c1 =0
  c2 = 0
  for i in range(picnum):
    img = x[:,i*256:(i+1)*256]
    img = (img - np.min(img))/(np.max(img) - np.min(img))
    #Img = Image.fromarray(np.uint8(img))
    if secSt <= i+1 <= secEn: #window size is 1sec
        if i+1 < 10:
            filename = 'time_seizure_000'+ str(i+1)
        elif 10 <= i+1 < 100:
            filename = 'time_seizure_00'+ str(i+1)
        elif 100 <= i+1 < 1000:
            filename = 'time_seizure_0'+ str(i+1)
        else:
            filename = 'time_seizure_'+ str(i+1)
        #np.save(path + name_F.split('.')[0] + '/' + filename+'.npy', img)
            #print('hi')
        c1+=1
    else:
        if i+1 < 10:
            filename = 'time_nonseizure_000'+ str(i+1)
        elif 10 <= i+1 < 100:
            filename = 'time_nonseizure_00'+ str(i+1)
        elif 100 <= i+1 < 1000:
            filename = 'time_nonseizure_0'+ str(i+1)
        else:
            filename = 'time_nonseizure_'+ str(i+1)
        
        #np.save(path + name_F.split('.')[0] + '/'+ filename+'.npy', img)
            #print('no=hi')
        c2+=1

  print(c1+c2)

In [7]:
seizure_trials, total_duration = createDataset()

START 

chb08_02.edf
2670
2841
3600
/content/drive/MyDrive/upsi-seizure/test1/chb08_02/
3600
3600
chb08_03.edf
chb08_04.edf
chb08_05.edf
2856
3046
3600
/content/drive/MyDrive/upsi-seizure/test1/chb08_05/
3600
3600
chb08_10.edf
chb08_11.edf
2988
3122
3600
/content/drive/MyDrive/upsi-seizure/test1/chb08_11/
3600
3600
chb08_12.edf
chb08_13.edf
2417
2577
3600
/content/drive/MyDrive/upsi-seizure/test1/chb08_13/
3600
3600
END 



In [19]:
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_curve, auc, fbeta_score

In [14]:
import random
random.seed(0)
x = random.sample(range(10), 10)
print(x)
print(x[:6], x[6:6+3], x[9:10])

[6, 9, 0, 2, 4, 3, 5, 1, 8, 7]
[6, 9, 0, 2, 4, 3] [5, 1, 8] [7]


In [11]:
import numpy as np
#b = np.zeros((2,2))
a = np.zeros((3,2,2))
b = np.append([[1],[2]],[[3],[4]], axis = 1)
a[0]= b
print(b)
print(a)

[[1 3]
 [2 4]]
[[[1. 3.]
  [2. 4.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]


In [32]:
import os
path = '/content/drive/MyDrive/upsi-seizure/test1/chb08_02'
print(len(os.listdir(path)))
path = '/content/drive/MyDrive/upsi-seizure/test1/chb08_05'
print(len(os.listdir(path)))
path = '/content/drive/MyDrive/upsi-seizure/test1/chb08_11'
print(len(os.listdir(path)))
path = '/content/drive/MyDrive/upsi-seizure/test1/chb08_13'
print(len(os.listdir(path)))
#path = '/content/drive/MyDrive/upsi-seizure/test1/chb08_02'
#print(len(os.listdir(path)))

3600
3600
3600
3600


In [41]:
name = 'chb08_02_time_nonseizure_2600.npy'
if '_nonseizure_'+str(1+1)+'.npy' in name:
  print('hi')

In [46]:
a = [[1,2], [3,4]]
b = [1,2]
c = np.stack((a,b) , axis = 1)
print(c)

ValueError: ignored

In [36]:
os.chdir(path)

In [50]:
create_feature_matrix(seizure_trials, path, 2, 3600)

chb08_02_time_nonseizure_2600.npy
1
1
features shape  (1800, 23, 512)
labels shape (1800,)


ValueError: ignored

In [7]:
sorted([0,2696,2697,2699,2698])

[0, 2696, 2697, 2698, 2699]

In [12]:
def last_4chars(x):
  #print(x[-4])
  return(x[-4])

import os
path = '/content/drive/MyDrive/upsi-seizure/test1/'
#print(sorted(os.listdir(path), key = last_4chars))
#last_4chars(os.listdir(path))

In [38]:
print(seizure_trials, total_duration)

['chb08_02.edf', 'chb08_05.edf', 'chb08_11.edf', 'chb08_13.edf'] 14400


In [138]:
seizure_trials, path, window_size, total_duration

(['chb08_02.edf', 'chb08_05.edf', 'chb08_11.edf', 'chb08_13.edf'],
 '/content/drive/MyDrive/upsi-seizure/test1/',
 4,
 14400)

In [159]:
for window in (range(0, total_duration, window_size)):
          print(window)

0
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
508
512
516
520
524
528
532
536
540
544
548
552
556
560
564
568
572
576
580
584
588
592
596
600
604
608
612
616
620
624
628
632
636
640
644
648
652
656
660
664
668
672
676
680
684
688
692
696
700
704
708
712
716
720
724
728
732
736
740
744
748
752
756
760
764
768
772
776
780
784
788
792
796
800
804
808
812
816
820
824
828
832
836
840
844
848
852
856
860
864
868
872
876
880
884
888
892
896
900
904
908
912
916
920
924
928
932
936
940
944
948
952
956
960
964
968
972
976
980
984
988
992
996
1000
1004
1008
1012
1016
102

In [8]:
for window in range(0,100,4):
  print(window)

0
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96


In [57]:
print(seizure_trials)
print(seizure_trials[0])
for trial in [seizure_trials[0]]:
  print(trial)
  print(path + trial.split('.')[0])

['chb08_02.edf', 'chb08_05.edf', 'chb08_11.edf', 'chb08_13.edf']
chb08_02.edf
chb08_02.edf
/content/drive/MyDrive/upsi-seizure/test1/chb08_02


In [80]:
x = np.zeros
x = np.append(x, np.array(np.load(path + trial.split('.')[0]+'/'+ name[0]))[0:23,0:255], axis = 1)
print(x.shape)

NameError: ignored

In [83]:
a = np.zeros
b = np.array([4, 5, 6])
np.stack((a, b))

ValueError: ignored

In [100]:
x = np.load('/content/drive/MyDrive/upsi-seizure/test1/chb08_02/time_nonseizure_0001.npy')
print(x)

[[0.45691203 0.51870138 0.51870138 ... 0.52812687 0.52393776 0.52453621]
 [0.52872531 0.51870138 0.51855177 ... 0.54862358 0.54727708 0.54024536]
 [0.51855177 0.51870138 0.51870138 ... 0.5136146  0.51032316 0.51256732]
 ...
 [0.52887493 0.51870138 0.51929982 ... 0.52618193 0.52378815 0.5149611 ]
 [0.53979653 0.51870138 0.51825254 ... 0.45302214 0.45706164 0.45227409]
 [0.54144225 0.51870138 0.51870138 ... 0.59395572 0.58916816 0.59141233]]


In [143]:
if '_seizure_' in 'time_seizure_2675.npy':
 print('hi')

hi


In [167]:
def create_feature_matrix(seizure_trials, path, window_size, total_duration):
  features = np.zeros(((int(total_duration/window_size)),23,256*window_size))
  labels = np.zeros((int(total_duration/window_size)),dtype=int)
  i = 0 #the feature index
 
  for trial in seizure_trials:
      name = sorted(os.listdir(path + trial.split('.')[0]), key= last_4chars)
      #print(name)
      for window in (range(0, int(total_duration/4), window_size)):
          #print(window)
          seizure = 0
          non_seizure = 0
          feature_prep = np.zeros((23,256*window_size))
          #feature_prep = np.zeros
          for j in range(window_size):
              if '_seizure_' in name[window+j]:
                  #print(name[window+j])
                  seizure+=1
                  #print('seizure no.', seizure)
                  #print('hi')
                  feature_prep[0:23,j*256:((j+1)*256)] = np.array(np.load(path + trial.split('.')[0]+'/'+ name[window+j]))[0:23,0:256]
              elif '_nonseizure_' in name[window+j]:
                  #print(name[window+j])
                  non_seizure+=1
                  #print('nonseizure no.', non_seizure)
                  #print('nohi')
                  feature_prep[0:23,j*256:((j+1)*256)] = np.array(np.load(path + trial.split('.')[0]+'/'+ name[window+j]))[0:23,0:256]
                  #print(feature_prep.shape)
          #print(feature_prep.shape)
          print('seizure no.', seizure)
          print('nonseizure no.', non_seizure)
          if seizure > non_seizure:
              #print('label should be 1')
              features[i] = feature_prep
              labels[i] = 1
              #print(features[i])
              print(labels[i])
          else:
              #print('label should be 0')
              features[i] = feature_prep
              labels[i] = 0
              #print(features[i])
              #print(labels[i])
          #print('================================')
          i = i+1
         # print(features)
         # print(features.shape)
          
   #       print(i)
                  

  #print(i)
    #print(features)
    #print(labels)
    #print(np.expand_dims(np.array(features), axis=3).shape)
    #print(labels.shape)
  print('features shape ', features.shape)
    # print('feature 0', features[0])
  print('labels shape', labels.shape)
  #for i in range(len(labels)):
    #print(labels[i])

  import torch
  import torch.utils as utils

  feature_tensor = torch.Tensor(features)
  feature_tensor = feature_tensor.unsqueeze(dim = 1)
  print('features tensorshape ', feature_tensor.shape)
  label_tensor = torch.Tensor(labels)

  dataset = utils.data.TensorDataset(feature_tensor, label_tensor)
  #dataloader = utils.data.DataLoader(dataset)
    #dataset = np.stack((features,labels), axis = 0)
  #print(dataset.shape)

  return dataset






In [168]:
import torch.utils.data as data
import torch
window_size = 4
path = '/content/drive/MyDrive/upsi-seizure/test1/'
dataset = create_feature_matrix(seizure_trials, path, window_size, total_duration)
#dataset = create_feature_matrix(seizure_trials, path, window_size, 3600)
#dataset_array = dataset.numpy()
dataset_array = np.array(dataset)
print(len(dataset))
train_size = int(0.7*len(dataset))
val_size = int(0.2*len(dataset))
test_size = int(0.1*len(dataset))

Streaming output truncated to the last 5000 lines.
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonseizure no. 4
seizure no. 0
nonse

In [148]:
if '_nonseizure_' in name[0]:
  print(0)

0


In [152]:
name = sorted(os.listdir(path + trial.split('.')[0]), key= last_4chars)
print(name[0])
print(type(name[0]))
print(len(name))

time_nonseizure_0001.npy
<class 'str'>
3600


In [158]:
def check_feature_matrix1(seizure_trials, path, window_size, total_duration):
  #features = np.zeros(((int(total_duration/window_size)),23,256*window_size))
  #labels = np.zeros((int(total_duration/window_size)),dtype=int)
  #i = 0 #the feature index
 
  for trial in seizure_trials:
      name = sorted(os.listdir(path + trial.split('.')[0]), key= last_4chars)
      print(name)
      for window in range(0,int(total_duration/4),window_size):
          #print(window)
          seizure = 0
          non_seizure = 0
          #feature_prep = np.zeros((23,256*window_size))
          for j in range(window_size):
              #print(window_size)
              #print('_seizure_'+str(window+j+1)+'.npy')
              if '_seizure_' in name[window+j]:
                  #print(name[window+j])
                  print('hi')
                  seizure+=1
                  #if window >= 3596:
                    #print(window+j+1)
                  #print('hi')
                  #feature_prep = np.append(np.array(np.load(name[window+j+1]))[0:23,:], axis = 1)
              elif '_nonseizure_' in name[window+j]:
                  #print(name[window+j])
                  print("========")
                  non_seizure+=1
                  #if window >= 3596:
                   # print(window+j+1)
                  #print('nohi')

  return 

In [202]:
seizure_trials

['chb08_02.edf', 'chb08_05.edf', 'chb08_11.edf', 'chb08_13.edf']

In [ ]:
path = '/content/drive/MyDrive/upsi-seizure/test1/'
check_feature_matrix1(seizure_trials, path, 4, total_duration)

In [114]:
dataset[0][1], torch.tensor(1.)

(tensor(0.), tensor(1.))

In [115]:
c=0
for i in range(len(dataset)):
  if dataset[0][1] == torch.tensor(1.):
    c=c+1
print(c)

0


In [ ]:
np.save(path+'preprocessed_data.npy',dataset,allow_pickle = True)

In [169]:
import numpy as np
#dataset = np.load(path+'preprocessed_data')
train_dataset, val_dataset, test_dataset = data.random_split(dataset,
                                               [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

In [122]:
len(train_dataset), len(val_dataset), len(test_dataset)

(630, 180, 90)

In [171]:
for i in range(len(test_dataset)):
  _, label = test_dataset[i][0], test_dataset[i][1]
  #print(f"Image tensor:\n{img}")
  #print(f"Image shape: {img.shape}")
  #print(f"Image datatype: {img.dtype}")
  print(f"Image label: {label}")
  #print(f"Label datatype: {type(label)}")

Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 1.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 0.0
Image label: 1

In [172]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler

weight = np.array((1,2))
weight[0] = 1
weight[1] = 23
samples_weight = np.array([weight[t] for t in range(2)])

samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

train_dataloader = DataLoader(dataset=train_dataset, # use custom created train Dataset
                                     #sampler = sampler,
                                     batch_size=32, # how many samples per batch?
                                     num_workers=4, # how many subprocesses to use for data loading? (higher = more)
                                     shuffle=False) # shuffle the data?

val_dataloader = DataLoader(dataset=val_dataset, # use custom created test Dataset
                                    #sampler = sampler,
                                    batch_size=32, 
                                    num_workers=4, 
                                    shuffle=False) # don't usually need to shuffle testing data

test_dataloader = DataLoader(dataset=test_dataset, # use custom created test Dataset
                                    #sampler = sampler,
                                    batch_size=32, 
                                    num_workers=4, 
                                    shuffle=False) # don't usually need to shuffle testing data


train_dataloader, val_dataloader, test_dataloader

# Get image and label from custom DataLoader
for i in range(len(train_dataloader)):
  img_custom, label_train = next(iter(train_dataloader))
  print(label_train)

img_custom_val, label_val = next(iter(val_dataloader))
#print(img_custom_val.shape)
print(label_train)
print(len(train_dataloader))
#for i in range(len(test_dataloader)):
img_custom_test, label_test = next(iter(test_dataloader))
print(img_custom_test.shape)
#print(label_test)

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
#print(f"Image shape : {img_custom.shape}")
#print(f"Label shape: {label_train.shape}")

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.

In [174]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [175]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

x = torch.randn(3,1,23,1024).to(device)
#print(x.shape)

class CNN(nn.Module):
    def __init__(self, size):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(1,8), stride = (1,1), padding = 'same')
        self.pool1 = nn.MaxPool2d(kernel_size=(1,8), stride=(1,8))
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(1,16), stride = (1,1), padding = 'same')
        self.pool2 = nn.MaxPool2d(kernel_size=(1,4), stride=(1,4))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1,8), stride = (1,1), padding = 'same')
        self.pool3 = nn.MaxPool2d(kernel_size=(1,4), stride=(1,4))
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(16,1), stride = (1,1), padding = 'same')
        self.pool4 = nn.MaxPool2d(kernel_size=(4,1), stride=(4,1))
        self.conv5 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(16,1), stride = (1,1), padding = 'same')
        self.pool5 = nn.MaxPool2d(kernel_size=(4,1), stride=(4,1))
        self.pool6 = nn.AvgPool2d(kernel_size=(1,int(size/128)), stride=(1,int(size/128)))
        self.fcn = nn.Linear(16, 2)
        #self.sigmoid=nn.Sigmoid()
    
    def forward(self,x):
        x=F.relu(self.conv1(x))
        #print('conv1 and relu', x.shape)
        x= self.pool1(x)
        #print('pool1', x.shape)
        x=F.relu(self.conv2(x))
        #print('conv2 and relu',x.shape)
        x= self.pool2(x)
        #print('pool2', x.shape)
        x=F.relu(self.conv3(x))
        #print('conv3 and relu', x.shape)
        x= self.pool3(x)
        #print('pool3', x.shape)
        x=F.relu(self.conv4(x))
        #print('conv4 and relu', x.shape)
        x= self.pool4(x)
        #print('pool4', x.shape)
        x=F.relu(self.conv5(x))
        #print('conv5 and relu', x.shape)
        x= self.pool5(x)
        #print('pool5', x.shape)
        x= self.pool6(x)
        #print('global avg pooling', x.shape)
        x = torch.squeeze(x)
        #print('squeezed x', x.shape)
        x= self.fcn(x)
        #print('fcn', x.shape)
        #x= self.sigmoid(x)
        #print('final prediction', x.shape)
        return x
    
torch.manual_seed(42)
#model_1 = CNN(256).to(device)
#model_2 = CNN(512).to(device)
model = CNN(1024).to(device)
#output = model_2(img_custom.to(device))
print(model)
#print(output.shape)
output = model(img_custom_test.to(device))
print(output)

CNN(
  (conv1): Conv2d(1, 4, kernel_size=(1, 8), stride=(1, 1), padding=same)
  (pool1): MaxPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 16, kernel_size=(1, 16), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 16, kernel_size=(1, 8), stride=(1, 1), padding=same)
  (pool3): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(16, 16, kernel_size=(16, 1), stride=(1, 1), padding=same)
  (pool4): MaxPool2d(kernel_size=(4, 1), stride=(4, 1), padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(16, 16, kernel_size=(16, 1), stride=(1, 1), padding=same)
  (pool5): MaxPool2d(kernel_size=(4, 1), stride=(4, 1), padding=0, dilation=1, ceil_mode=False)
  (pool6): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
  (fcn): Linear(in_features=16, out_features=2, bias=True)
)
tensor([[0.2

In [ ]:
def weighted_binary_cross_entropy(output, target, weights=None):
        
    if weights is not None:
        assert len(weights) == 2
        
        loss = weights[1] * (target * torch.log(output)) + \
               weights[0] * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))

In [176]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    
    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X = X.to(device)
        y = y.type(torch.LongTensor) 
        y = y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [47]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 31.4 MB/s 


In [177]:
from torchmetrics import Specificity
def val_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval() 
    
    # Setup test loss and test accuracy values
    val_loss, val_acc, val_spec, val_avg_spec = 0, 0, 0, 0
    
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X = X.to(device)
            y = y.type(torch.LongTensor) 
            y = y.to(device)
    
            # 1. Forward pass
            val_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(val_pred_logits, y)
            val_loss += loss.item()
            
            # Calculate and accumulate accuracy
            val_pred_labels = val_pred_logits.argmax(dim=1)
            val_acc += ((val_pred_labels == y).sum().item()/len(val_pred_labels))

            # Calculate and accumulate specificity
            val_pred_labels = val_pred_logits.argmax(dim=1)
            print(val_pred_labels)
            print(y)
            specificity = Specificity(average='micro', num_classes=2).to(device)
            val_spec += specificity(val_pred_labels, y)
        #val_avg_spec = val_spec/batch-1   
            
    # Adjust metrics to get average loss and accuracy per batch 
    val_loss = val_loss / len(dataloader)
    val_acc = val_acc / len(dataloader)
    val_avg_spec = val_spec / len(dataloader)
    return val_loss, val_acc, val_avg_spec

In [178]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          val_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
        "val_spec": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        val_loss, val_acc, val_avg_spec = val_step(model=model,
            dataloader=val_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"val_loss: {val_loss:.4f} | "
            f"val_acc: {val_acc:.4f} |"
            f"val_spec: {val_avg_spec:.4f} "
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)
        results["val_spec"].append(val_avg_spec)

    # 6. Return the filled results at the end of the epochs
    return results

In [179]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 10

# Recreate an instance of TinyVGG
model = CNN(1024).to(device)
weight = torch.tensor([1,23])
#weight = weight.type(torch.LongTensor)
weight = weight.to(device)
# Setup loss function and optimizer
#loss_fn = nn.BCELoss(weight = weight)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, betas=(0.9, 0.999))

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_results = train(model=model, 
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/10 [00:00<?, ?it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

In [180]:
test_loss, test_acc, test_spec = val_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
print(
      f"test_loss: {test_loss:.4f} | "
      f"test_acc: {test_acc:.4f} | "
      f"test_spec: {test_spec:.4f} ")

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0,

In [56]:
print(test_dataloader.shape)

AttributeError: ignored

In [1]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
try: 
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo
    
from torchinfo import summary
summary(model, input_size=[1, 1, 23, 512])

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [2]                       --
├─Conv2d: 1-1                            [1, 4, 23, 512]           36
├─MaxPool2d: 1-2                         [1, 4, 23, 64]            --
├─Conv2d: 1-3                            [1, 16, 23, 64]           1,040
├─MaxPool2d: 1-4                         [1, 16, 23, 16]           --
├─Conv2d: 1-5                            [1, 16, 23, 16]           2,064
├─MaxPool2d: 1-6                         [1, 16, 23, 4]            --
├─Conv2d: 1-7                            [1, 16, 23, 4]            4,112
├─MaxPool2d: 1-8                         [1, 16, 5, 4]             --
├─Conv2d: 1-9                            [1, 16, 5, 4]             4,112
├─MaxPool2d: 1-10                        [1, 16, 1, 4]             --
├─AvgPool2d: 1-11                        [1, 16, 1, 1]             --
├─Linear: 1-12                           [2]                       34
Tot

In [25]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 15.2 MB/s 


In [32]:
from torchmetrics import ConfusionMatrix, Specificity

_, target = next(iter(test_dataloader))
print(target.shape)
print(test_pred.shape)

confmat = ConfusionMatrix(num_classes=2)
confmat(test_pred, target.long())
specificity = Specificity(average='macro', num_classes=2)
specificity(test_pred, target.long())


torch.Size([32])
torch.Size([20])


ValueError: ignored

In [ ]:
conf_matrix = torch.zeros(2, 2)
for t, p in zip(target, test_pred):
    conf_matrix[t, p] += 1

print('Confusion matrix\n', conf_matrix)

TP = conf_matrix.diag()
for c in range(nb_classes):
    idx = torch.ones(nb_classes).byte()
    idx[c] = 0
    # all non-class samples classified as non-class
    TN = conf_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
    # all non-class samples classified as class
    FP = conf_matrix[idx, c].sum()
    # all class samples not classified as class
    FN = conf_matrix[c, idx].sum()
    
    print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
        c, TP[c], TN, FP, FN))

In [77]:
def generatePathList(patients, val_size, test_size):
    parent_path = '/content/drive/MyDrive/upsi-seizure/test/'
    pathList = []
    for indexPatient in range(0, len(patients)):
        sub_path = 'chb'+patients[indexPatient]+'/'
        directory_name = parent_path
        for filename in os.listdir(directory_name):
            pathList.append(directory_name+filename)
    L = len(pathList)
    train_index_size = int(L*(1-val_size-test_size))
    val_index_size = int(L*val_size)
    test_index_size = int(L*test_size)
    ##############CORRECT THIS AND RANDOMISE####################
    #train_index = random.sample(pathList, train_index_size)
    #val_index = random.sample(pathList, val_index_size)
    #test_index = random.sample(pathList, test_index_size)
    #print(test_index)
    random.seed(0)
    index = random.sample(range(L), L)
    return index[:train_index_size],index[train_index_size:train_index_size+val_index_size], index[train_index_size+val_index_size:L],pathList
    #return pathList.index(train_index), pathList(val_index), pathList.index(test_index), pathList

class DataGenerator(Sequence):

    def __init__(self, seizure_trials, index, window_size, total_duration, pathList, parent_path = '/content/drive/MyDrive/upsi-seizure/test/', batch_size=32):
        self.batch_size = batch_size
        self.parent_path = parent_path
        self.pathList = pathList
        self.index = index
        self.L = len(self.index)
        self.window_size = window_size
        self.total_duration = total_duration
        self.seizure_trials = seizure_trials


    def __len__(self):
        return self.L - self.batch_size

    def __getitem__(self, seizure_trials, idx,window_size, total_duration):
        batch_indexs = self.index[idx:(idx+self.batch_size)]
        image_path = [self.pathList[k] for k in batch_indexs]
        #print(batch_indexs)
        #print(image_path)
        return self._load_image(image_path, seizure_trials, window_size, total_duration)

  
    def _load_image(self, image_path, seizure_trials, window_size, total_duration):
        features = np.zeros(((len(image_path)),23,256*window_size))
        labels = np.zeros((len(image_path)),dtype=int)
        i = 0 #the feature index
        for trial in seizure_trials:
            for name,window in zip((image_path), (range(0, window_size, total_duration))):
            #print(name)
                seizure = 0
                non_seizure = 0
                feature_prep = np.zeros((23,256*window_size))
                for j in range(window_size):
                    if '_seizure_'+str(window+j+1) in name:
                        seizure+=1
                        feature_prep = np.append(np.array(np.load(name))[0:23,:], axis = 1)
                    elif '_nonseizure_'+str(window+j+1) in name:
                        non_seizure+=1
                        feature_prep = np.append(np.array(np.load(name))[0:23,:], axis = 1)
                if seizure > non_seizure:
                    features[i] = feature_prep
                    labels[i] = 1
                else:
                    features[i] = feature_prep
                    labels[i] = 0
                i = i+1
                print(i)
                    

        print(i)
        #print(features)
        #print(labels)
        #print(np.expand_dims(np.array(features), axis=3).shape)
        #print(labels.shape)
        print('features shape ', features.shape)
       # print('feature 0', features[0])
        print('labels shape', labels.shape)
       # print('label 0', labels[0])
        #print(features.shape)
        s = np.expand_dims(np.array(features), axis=3)
        #print(s.shape)
        return np.expand_dims(np.array(features), axis=3),labels

In [71]:
import os

print(len(os.listdir(r"/content/drive/MyDrive/upsi-seizure/test")))

3600


In [10]:
import tensorflow
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Activation, Flatten, LSTM, Bidirectional, ConvLSTM2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, BatchNormalization, TimeDistributed, SimpleRNN, GRU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.backend import clear_session

In [11]:
model = Sequential()

model.add(Conv2D(4, (1, 8), strides = (1,1), padding = 'same', input_shape=((23, 256, 1))))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 8), strides=(1,8)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (1, 16),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 4), strides=(1,4)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (1, 8),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 4), strides=(1,4)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (16, 1),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 1), strides=(4,1)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (16, 1),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 1), strides=(4,1)))
#model.add(Dropout(0.15))

model.add(GlobalAveragePooling2D())

model.add(Dense(1, activation='sigmoid'))

sgd = SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(metrics=['accuracy',keras.metrics.Recall(name='sen'), keras.metrics.SpecificityAtSensitivity(0.98)], 
              loss='binary_crossentropy', optimizer= 'sgd')

In [78]:
import os
import random
import collections
train_id, val_id, test_id, path_list = generatePathList(patients, val_size= 0.3, test_size=0.1) # 30% test_size 
#print(test_id)
#print(train_id)
window_size = 2
train_data = DataGenerator(seizure_trials,train_id,  window_size, trial_duration, path_list)
val_data = DataGenerator(seizure_trials,val_id, window_size, trial_duration, path_list)
test_data  =  DataGenerator(seizure_trials,test_id, window_size, trial_duration, path_list)

In [78]:
#test_data.__getitem__(100)

In [54]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

2128
1048
328


In [79]:
train_data.__getitem__(seizure_trials, 0, 2, 3600)

1
1
features shape  (32, 23, 512)
labels shape (32,)


(array([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
      

In [32]:
import numpy as np

In [16]:
history = model.fit_generator(generator=train_data, epochs= 2, verbose=1, steps_per_epoch=None, 
                    validation_data=test_data, class_weight={0:1, 1:20}, workers=4, use_multiprocessing=True,
                    shuffle=False, initial_epoch=0)

TypeError: ignored

In [63]:
y_true= [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(y_true)
print(y_pred)

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.17241904139518738, 0.9545454382896423, 0.6899086833000183]


In [62]:
confusion_matrix(y_true, y_pred)

ValueError: ignored

In [36]:
print(history)

In [66]:
y_pred = model.evaluate(test_data, batch_size = 32)

[1111, 3155, 1138, 16148, 5963, 11390, 925, 8960, 15675, 1890, 4611, 16218, 12759, 2456, 9913, 14993, 3668, 17427, 10137, 665, 2628, 3781, 12491, 5656, 9605, 8445, 10252, 2246, 2192, 6386, 17392, 10180]
['/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_1112.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_3156.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_1139.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_21_time_nonseizure_1749.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_05_time_nonseizure_2364.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_13_time_nonseizure_591.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_926.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_11_time_nonseizure_1761.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_21_time_nonseizure_1276.npy', '/content/drive/MyDrive/upsi-seizure/norma

ValueError: ignored